In [1]:
import numpy as np
from numpy import linalg as lg
import pandas as pd
import copy

In [2]:
while True:
    choose = input("File: f // Assumption: a // manually: m // \t")
    if choose == 'f' or 'a' or 'm':
        break

File: f // Assumption: a // manually: m // 	a


In [3]:
# File
if choose == 'f':
    file_first_A =  "A.txt"
    file_first_b =  "b.txt"

    file_second_A = "A2.txt"
    file_second_b = "b2.txt"

    while True:
        file_name = input('if test file_first Enter: f1 // if test file_second Enter: f2 // Choose:  ')
        if file_name == 'f1':
            file_name_A = file_first_A
            file_name_b = file_first_b
            break
        if file_name == 'f2':
            file_name_A = file_second_A
            file_name_b = file_second_b
            break

    file_A = open(file_name_A)
    line1 = file_A.readlines()
    df_A = pd.DataFrame()
    for i in range(1, int(line1[0].split()[0])+1):
        df_A[i-1] = line1[i].split()
    df_A = df_A.T.astype('float')
    # print(df1_A)

    file_b = open(file_name_b)
    line2 = file_b.readlines()
    df_b = pd.DataFrame()
    for j in range(int(line1[0].split()[0])):
        df_b[j] = line2[j].split()
    df_b.index = [int(line1[0].split()[0])]
    df_b = df_b.T.astype('float')
    # print(df1_b)
    
    A_tmp_c = df_A.to_numpy()
    b_tmp_c = df_b.to_numpy().reshape(-1)
    A_tmp_p = df_A.to_numpy()
    b_tmp_p = df_b.to_numpy().reshape(-1)
    
elif choose == 'm':
    n = int(input('shape of matrix: '))
    A_tmp = np.zeros((n, n))
    b_tmp = np.zeros(n)
    for i in range(n):
        for j in range(n):
            print('A' + str(i) + str(j), end=' ')
            A_tmp[i][j] = float(input(''))
            
        print('b' + str(i) , end=' ')
        b_tmp[i] = float(input(''))
        print(A_tmp[i,:], b_tmp[i])
    A_tmp_c = A_tmp
    b_tmp_c = b_tmp
    A_tmp_p = A_tmp
    b_tmp_p = b_tmp
    
else:
    A_tmp_p = np.array([[ 2, 6, -2],
                  [-4, 4,  2],
                  [ 1, 2,  3]])
    b_tmp_p = np.array([6, 2, 6])
    
    
    A_tmp_c = np.array([[0, 1, 1],
                      [1, 2, 3],
                      [1, 1, 1]])
    b_tmp_c = np.array([2, 6, 3])


# EconomicPartialPivoting

In [4]:
# -------------------------------------#
#          economic                    #
#            MA=U                      #
# -------------------------------------#
file_p = open('EconomicPartialPivoting', 'w')


n, _ = np.shape(A_tmp_p)
A = np.zeros(np.shape(A_tmp_p))
b = np.zeros(n)
A[:, :] = A_tmp_p[:, :]
b[:] = b_tmp_p[:].copy()

row = np.eye(1, n - 1)

for k in range(0, n - 1):
    r = k
    Max = np.abs(A[k, k])
    for i in range(k + 1, n):
        if np.abs(A[i, k]) > Max:
            Max = np.abs(A[i, k])
            r = i
    row[0][k] = r

    f = A[r, k:].copy()
    A[r, k:] = A[k, k:].copy()
    A[k, k:] = f.copy()

    
    for i in range(k + 1, n):
        A[i, k] = -(A[i, k] / A[k, k])
        for j in range(k + 1, n):
            A[i, j] = A[i, j] + A[i, k] * A[k, j]
#         b[i] = b[i] + A[i, k] * b[k]  # b استاد من هرچی سعی کردم نتونستم جواب
                                        #!!! رو با این الگوریتم  درست بدست بیارم   

U = np.triu(A)

P_stack = np.zeros((n - 1, n, n))
M_stack = np.zeros((n - 1, n, n))

for k in range(n - 1):
    P_stack[k, :, :] = np.eye(n)
    f = P_stack[k, int(row[0][k]), :].copy()
    P_stack[k, int(row[0][k]), 0:] = P_stack[k, k, :].copy()
    P_stack[k, k, :] = f.copy()

    M_stack[k, :, :] = np.eye(n)
    for i in range(k + 1, n):
        M_stack[k, i, k] = A[i, k]

M = np.eye(n, n)
for k in range(0, n - 1):
    T = P_stack[k, :, :].dot(M)
    M = M_stack[k, :, :].dot(T)

for k in range(1, n):
    file_p.write('P' + str(k) + ':\n')
    file_p.write(str(P_stack[k - 1, :, :]))
    file_p.write('\nM' + str(k) + ':\n')
    file_p.write(str(M_stack[k - 1, :, :]))
    file_p.write('\n==========================================\n')
    print('P' + str(k) + ':\n', P_stack[k - 1, :, :])
    print('\nM' + str(k) + ':\n', M_stack[k - 1, :, :])
    print('==========================================\n')

# ==================================================== #

# ==================================================== #
P = np.eye(n, n)
for k in range(0, n - 1):
    P = P_stack[k, :, :].dot(P)

L = P.dot(np.linalg.inv(M))
file_p.write('P:\n')
file_p.write(str(P))
file_p.write('\nM:\n')
file_p.write(str(M))
file_p.write('\nU:\n')
file_p.write(str(U))
print('P:\n', P)
print('\nM:\n', M)
print('\nU:\n', U)
print('\nA:\n', A)
# print('\n==========================================\n')

b = M.dot(b)
x = np.zeros(n)
for i in reversed(range(n)):
    x[i] = b[i]
    for j in range(i+1, n):
        x[i] = x[i] - A[i][j] * x[j]
    x[i] = x[i] / A[i][i]

for k in range(n):
    file_p.write('\nx' + str(k) + ' = ')
    file_p.write(str(x[k]))
    print('x' + str(k) + ' = ', x[k])


file_p.close()

P1:
 [[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]

M1:
 [[1.   0.   0.  ]
 [0.5  1.   0.  ]
 [0.25 0.   1.  ]]

P2:
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

M2:
 [[ 1.     0.     0.   ]
 [ 0.     1.     0.   ]
 [ 0.    -0.375  1.   ]]

P:
 [[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]

M:
 [[ 0.      1.      0.    ]
 [ 1.      0.5     0.    ]
 [-0.375   0.0625  1.    ]]

U:
 [[-4.     4.     2.   ]
 [ 0.     8.    -1.   ]
 [ 0.     0.     3.875]]

A:
 [[-4.     4.     2.   ]
 [ 0.5    8.    -1.   ]
 [ 0.25  -0.375  3.875]]
x0 =  1.0
x1 =  1.0
x2 =  1.0


In [5]:
# -------------------------------------#
#              economic               #
#            MAQ=U & PAQ=LU             #
# -------------------------------------#
file_c = open('DecompositionCompletePivoting', 'w')

n, _ = np.shape(A_tmp_c)

A = np.zeros(np.shape(A_tmp_c))
b = np.zeros(n)

A[:, :] = A_tmp_c[:, :]
b[:] = b_tmp_c[:].copy()

row = np.eye(1, n - 1)
col = np.eye(1, n - 1)

for k in range(0, n - 1):
    r = k
    s = k
    Max = np.abs(A[k, k])
    for j in range(k, n):
        for i in range(k, n):
            if np.abs(A[i, j]) > Max:
                Max = np.abs(A[i,j])
                r = i
                s = j
                
    row[0][k] = r
    col[0][k] = s
    
    f_row   = A[k, :].copy()
    A[k, :] = A[r, :].copy()
    A[r, :] = f_row.copy()

    f_col   = A[:, k].copy()
    A[:, k] = A[:, s].copy()
    A[:, s] = f_col.copy()
    
    for i in range(k + 1, n):
        A[i, k] = A[i, k] / A[k, k]
        for j in range(k + 1, n):
            A[i, j] = A[i, j] - A[i, k] * A[k, j]

P_stack = np.zeros((n - 1, n, n))
M_stack = np.zeros((n - 1, n, n))
Q_stack = np.zeros((n - 1, n, n))

for k in range(n - 1):
    P_stack[k, :, :] = np.eye(n)
    f1 = P_stack[k, int(row[0][k]), :].copy()
    P_stack[k, int(row[0][k]), 0:] = P_stack[k, k, :].copy()
    P_stack[k, k, :] = f1.copy()
    
    Q_stack[k, :, :] = np.eye(n)
    f2 = Q_stack[k, :, int(col[0][k])].copy() 
    Q_stack[k, 0:, int(col[0][k])] = Q_stack[k, :, k].copy()
    Q_stack[k, :, k] = f2.copy()
    
    M_stack[k, :, :] = np.eye(n)
    for i in range(k + 1, n):
        M_stack[k, i, k] = -A[i, k]

M = np.eye(n, n)
for k in range(0, n - 1):
    T = P_stack[k, :, :].dot(M)
    M = M_stack[k, :, :].dot(T)

for k in range(1, n):
    file_c.write('P' + str(k) + ':\n')
    file_c.write(str(P_stack[k - 1, :, :]))
    file_c.write('Q' + str(k) + ':\n')
    file_c.write(str(Q_stack[k - 1, :, :]))
    file_c.write('\nM' + str(k) + ':\n')
    file_c.write(str(M_stack[k - 1, :, :]))
    file_c.write('\n==========================================\n')
    print('P' + str(k) + ':\n', P_stack[k - 1, :, :])
    print('\nQ' + str(k) + ':\n', Q_stack[k - 1, :, :])
    print('\nM' + str(k) + ':\n', M_stack[k - 1, :, :])
    print('==========================================')

P = np.eye(n, n)
Q = np.eye(n, n)
for k in range(0, n - 1):
    P = P_stack[k, :, :].dot(P)
    Q = Q.dot(Q_stack[k, :, :])

U = np.triu(A)

# # ==================================================== #
file_c.write('P:\n')
file_c.write(str(P))
file_c.write('\nQ:\n')
file_c.write(str(Q))
file_c.write('\nM:\n')
file_c.write(str(M))
file_c.write('\nU:\n')
file_c.write(str(U))
file_c.write('\nA:\n')
file_c.write(str(A))
file_c.write('\n==========================================\n')
print('P:\n', P)
print('\nQ:\n', Q)
print('\nM:\n', M)
print('\nU:\n', U)
print('\nA:\n', A)
print('==========================================')
# # ==================================================== #

# bb = b.copy()
# for k in range(n-1):                  # همینطور در این قسمت استاد
#     b_old = bb[k].copy()              # b رو نتونستم 
#     bb[k] = bb[int(row[0][k])].copy() # !!! با این الگوریتم بدست بیارم
#     bb[int(row[0][k])] = b_old
#     for i in range(k+1, n):
#         bb[i] = bb[i] + A[i][k] * b[k]

bb = M.dot(b)

y = np.zeros(n)    
for i in reversed(range(n)):
    y[i] = bb[i]
    for j in range(i+1, n):
        y[i] = y[i] - A[i][j] * y[j]
    y[i] = y[i] / A[i][i]
    
x = np.zeros(n)
x[:] = y[:].copy()
for i in reversed(range(n-1, 1)):
    x_old = x[i].copy()
    x[i] = x[int(col[0][i])]
    x[int(col[0][i])] = x_old 

for k in range(n):
    file_c.write('\nx' + str(k) + ' = ')
    file_c.write(str(np.round_(x[k])))
    print('x' + str(k) + ' = ', np.round_(x[k]))

file_c.close()

P1:
 [[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]

Q1:
 [[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]

M1:
 [[ 1.          0.          0.        ]
 [-0.33333333  1.          0.        ]
 [-0.33333333  0.          1.        ]]
P2:
 [[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]

Q2:
 [[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]

M2:
 [[1.  0.  0. ]
 [0.  1.  0. ]
 [0.  0.5 1. ]]
P:
 [[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]

Q:
 [[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]

M:
 [[ 0.          1.          0.        ]
 [ 0.         -0.33333333  1.        ]
 [ 1.         -0.5         0.5       ]]

U:
 [[3.         1.         2.        ]
 [0.         0.66666667 0.33333333]
 [0.         0.         0.5       ]]

A:
 [[ 3.          1.          2.        ]
 [ 0.33333333  0.66666667  0.33333333]
 [ 0.33333333 -0.5         0.5       ]]
x0 =  1.0
x1 =  1.0
x2 =  1.0
